<a href="https://colab.research.google.com/github/Peggy4444/hello-world/blob/master/Behavioral_cloning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [145]:
import numpy
#from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
numpy.random.seed(7)
from sklearn.model_selection import train_test_split

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 18390068408633566487, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 13839852268217439739
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [4]:
import pandas as pd
import numpy as np

In [5]:
from google.colab import files
uploaded = files.upload()

Saving Bayern_features.xlsx to Bayern_features.xlsx


In [6]:
import io
df_features = pd.read_excel('Bayern_features.xlsx')

In [7]:
df_features

,Unnamed: 0,period_id,bodypart_id,type_id,result_id,start_x_norm,start_y_norm,end_x_norm,end_y_norm,start_distance_to_goal,start_angle_to_goal,end_distance_to_goal,end_angle_to_goal,clearance,corner_crossed,corner_short,cross,dribble,foul,freekick_crossed,freekick_short,goalkick,interception,keeper_save,pass,shot,shot_freekick,shot_penalty,tackle,take_on,throw_in,diff_x,diff_y,distance_covered,time_played,time_remaining
0,0,1,0,0,1,0.50,0.50,0.50,0.48,52.500000,0.000000,52.517612,38.602941,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0.00,-1.36,1.360000,2.409746,2732
1,1,1,0,0,1,0.50,0.48,0.78,0.22,52.517612,38.602941,29.935457,1.213235,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,29.40,-17.68,34.306594,2.506082,2732
2,2,1,0,0,1,0.78,0.22,0.94,0.46,29.935457,1.213235,6.862099,2.316176,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,16.80,16.32,23.421836,6.946706,2728
3,3,1,0,0,1,0.94,0.46,0.80,0.10,6.862099,2.316176,34.363353,0.772059,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-14.70,-24.48,28.554516,10.786491,2724
4,4,1,0,0,1,0.80,0.10,0.73,0.04,34.363353,0.772059,42.215648,0.906330,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-7.35,-4.08,8.406480,12.684514,2722
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45659,45659,2,0,0,1,0.35,0.19,0.29,0.33,71.431288,3.237666,75.440944,6.448962,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-6.30,9.52,11.415796,5554.025067,9
45660,45660,2,0,7,1,0.29,0.33,0.22,0.16,75.440944,6.448962,85.100790,3.542388,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-7.35,-11.56,13.698763,5556.882300,7
45661,45661,2,0,0,1,0.22,0.16,0.17,0.06,85.100790,3.542388,92.142981,2.912767,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-5.25,-6.80,8.590838,5559.417105,4
45662,45662,2,0,0,1,0.17,0.06,0.33,0.06,92.142981,2.912767,76.448211,2.351270,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,16.80,0.00,16.800000,5561.358967,2


In [8]:
uploaded = files.upload()

Saving Bayern_actions.xlsx to Bayern_actions.xlsx


In [9]:
Bayern_actions = pd.read_excel('Bayern_actions.xlsx')

In [12]:
End_list_index=[]
for index, row in Bayern_actions.iterrows():
    if (Bayern_actions['short_team_name'].iloc[index] == Bayern_actions['short_team_name'].shift(1).iloc[index]) and (Bayern_actions['short_team_name'].iloc[index] == Bayern_actions['short_team_name'].shift(2).iloc[index]) and (Bayern_actions['short_team_name'].iloc[index] != Bayern_actions['short_team_name'].shift(-1).iloc[index]) and (Bayern_actions['short_team_name'].iloc[index] != Bayern_actions['short_team_name'].shift(-2).iloc[index]) and (Bayern_actions['short_team_name'].iloc[index] != Bayern_actions['short_team_name'].shift(-3).iloc[index]):
        End_list_index.append(index)

In [14]:
len(End_list_index)

2472

In [82]:
Bayern_End_list_index=[]
for i in End_list_index:
    if Bayern_actions['short_team_name'].iloc[i]== "Bayern München" :
        Bayern_End_list_index.append(i)

In [83]:
len(Bayern_End_list_index)

1218

In [18]:
Apponent_End_list_index=[]
for i in End_list_index:
    if Bayern_actions['short_team_name'].iloc[i]!= "Bayern München" :
        Apponent_End_list_index.append(i)

In [20]:
len(Apponent_End_list_index)

1254

In [84]:
End_list_index[:10]

[7, 37, 55, 61, 75, 79, 82, 130, 154, 164]

In [80]:
Bayern_End_list_index[:10]

[37, 55, 79, 164, 197, 254, 278, 292, 313, 343]

In [79]:
Apponent_End_list_index[:10]


[7, 61, 75, 82, 130, 154, 263, 287, 297, 326]

In [21]:
def bayern_possession_list_maker(Bayern_actions, Bayern_End_list_index, Apponent_End_list_index):
    all_possessions=[]
    for delays in range(1,len(Bayern_End_list_index)-3):
        df= df_features.iloc[Apponent_End_list_index[delays-1]:Bayern_End_list_index[delays+2]]
        pos= df.to_numpy()
        all_possessions.append(pos)
    return all_possessions

In [24]:
Bayern_possessions= bayern_possession_list_maker(Bayern_actions, Bayern_End_list_index, Apponent_End_list_index)

In [76]:
del Bayern_possessions[0]

In [77]:
len(Bayern_possessions)

1214

In [194]:
def Apponent_possession_list_maker(Bayern_actions, Bayern_End_list_index, Apponent_End_list_index):
    all_possessions2=[]
    for delays in range(0,len(Bayern_End_list_index)-1):
        df= df_features.iloc[Bayern_End_list_index[delays]+1:Apponent_End_list_index[delays]+2]
        pos= df.to_numpy()
        all_possessions2.append(pos)
    return all_possessions2

In [195]:
Apponent_possessions= Apponent_possession_list_maker(Bayern_actions, Bayern_End_list_index, Apponent_End_list_index)

In [196]:
len(Apponent_possessions)

1217

In [41]:
def offensive_label_list_maker(Bayern_actions, Bayern_End_list_index):
    possessions_label_offensive=[]
    for i in Bayern_End_list_index:
        off_labels= Bayern_actions['type_name'].iloc[i]
        possessions_label_offensive.append(off_labels)
    return possessions_label_offensive


In [43]:
offensive_possession_labels=offensive_label_list_maker(Bayern_actions, Bayern_End_list_index)

In [45]:
len(offensive_possession_labels)

1218

In [54]:
#offensive_possession_labels

In [49]:
Bayern_actions['short_team_name'].unique()

array(['Bayer Leverkusen', 'Bayern München', 'Augsburg',
       'Eintracht Frankfurt', "Borussia M'gladbach", 'Köln',
       'Werder Bremen', 'Hoffenheim', 'Hannover 96', 'Stuttgart',
       'Mainz 05', 'Schalke 04', 'Wolfsburg', 'Hertha BSC', 'Freiburg',
       'Hamburger SV', 'RB Leipzig', 'Borussia Dortmund'], dtype=object)

In [66]:
del Apponent_End_list_index[-1]

In [68]:
def defensive_label_list_maker(Bayern_actions, Apponent_End_list_index):
    possessions_label_defensive=[]
    for i in Apponent_End_list_index:
        labels_defensive= Bayern_actions['type_name'].iloc[i+1]
        possessions_label_defensive.append(labels_defensive)
    return possessions_label_defensive

In [134]:
defensive_possession_labels=defensive_label_list_maker(Bayern_actions, Apponent_End_list_index)

In [70]:
#defensive_possession_labels


In [67]:
max(Apponent_End_list_index)

45644

In [109]:
print(len(offensive_possession_labels), len(Bayern_possessions))

1214 1214


In [197]:
print(len(defensive_possession_labels), len(Apponent_possessions))

1217 1217


In [111]:
off_possession_label_array= np.array(offensive_possession_labels).T
type(off_possession_label_array)

numpy.ndarray

In [178]:
def_possession_label_array= np.array(defensive_possession_labels).T
type(def_possession_label_array)

numpy.ndarray

In [113]:
all_Bayern_possessions_array= np.array(Bayern_possessions)
all_Bayern_possessions_array.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(1214,)

In [198]:
all_Apponent_possessions_array= np.array(Apponent_possessions)
all_Apponent_possessions_array.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


(1217,)

In [115]:
Bayern_ending_actions = list(set(offensive_possession_labels))

In [116]:
Bayern_ending_actions

['pass',
 'tackle',
 'foul',
 'clearance',
 'take_on',
 'dribble',
 'interception',
 'corner_crossed',
 'throw_in',
 'cross',
 'shot']

In [135]:
Apponent_ending_actions = list(set(defensive_possession_labels))

In [136]:
Apponent_ending_actions

['pass',
 'tackle',
 'freekick_short',
 'interception',
 'corner_crossed',
 'goalkick',
 'dribble',
 'clearance',
 'take_on',
 'throw_in',
 'keeper_save',
 'cross',
 'corner_short',
 'shot']

In [119]:
for index, item in enumerate(offensive_possession_labels):
    if item == 'shot':
        offensive_possession_labels[index] = 1
    if item != 'shot':
        offensive_possession_labels[index] = 0

In [120]:
list(set(offensive_possession_labels))

[0, 1]

In [137]:
for index, item in enumerate(defensive_possession_labels):
    if item == 'tackle':
        defensive_possession_labels[index] = 0
    elif item == 'clearance':
        defensive_possession_labels[index] = 1
    elif item == 'interception':
        defensive_possession_labels[index] = 2
    else:
        defensive_possession_labels[index] = 3
        

In [138]:
list(set(defensive_possession_labels))

[0, 1, 2, 3]

In [139]:
offensive_possession_labels_array= np.array(offensive_possession_labels).T

In [140]:
offensive_possession_labels_array

array([0, 0, 1, ..., 0, 0, 0])

In [183]:
defensive_possession_labels_array= np.array(defensive_possession_labels).T

In [185]:
len(defensive_possession_labels_array)

1217

In [142]:
defensive_possession_labels_array

array([2, 3, 3, ..., 2, 3, 1])

In [143]:
print(tf.__version__)


2.4.1


Offensive network:

In [216]:
X_train, X_test, y_train, y_test = train_test_split(all_Bayern_possessions_array, offensive_possession_labels_array)

In [217]:
X_train = sequence.pad_sequences(X_train, maxlen=10)
X_test = sequence.pad_sequences(X_test, maxlen=10)

In [218]:
from keras.utils import to_categorical
y_train= to_categorical(y_train)
y_test= to_categorical(y_test)


In [219]:
print(X_train.shape,y_train.shape)

(910, 10, 36) (910, 2)


In [220]:
print(X_test.shape, y_test.shape)

(304, 10, 36) (304, 2)


In [226]:
%%time
from keras.layers.convolutional import Conv1D 
#from keras.layers import Conv1D, MaxPoling1D
off_model = Sequential()
off_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(10,36)))
#model.add(MaxPooling1D(pool_size=1))
off_model.add(LSTM(100))
off_model.add(Dense(2, activation='sigmoid'))
off_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(off_model.summary())
off_model.fit(X_train, y_train, epochs=10, batch_size=64)
# Final evaluation of the model
scores = off_model.evaluate(X_test, y_test, verbose=0)
print(off_model.summary())
# Final evaluation of the model
print("accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_21 (Conv1D)           (None, 8, 32)             3488      
_________________________________________________________________
lstm_20 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_20 (Dense)             (None, 2)                 202       
Total params: 56,890
Trainable params: 56,890
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
15/15 [==============================] - 1s 5ms/step - loss: 0.5764 - accuracy: 0.7470
Epoch 2/10
15/15 [==============================] - 0s 4ms/step - loss: 0.4191 - accuracy: 0.8632
Epoch 3/10
15/15 [==============================] - 0s 4ms/step - loss: 0.4336 - accuracy: 0.8604
Epoch 4/10
15/15 [==============================] - 0s 4ms/step - loss: 0.4676 -

Defensive network:

In [199]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(all_Apponent_possessions_array, defensive_possession_labels_array)

In [202]:
X_train1 = sequence.pad_sequences(X_train1, maxlen=10)
X_test1 = sequence.pad_sequences(X_test1, maxlen=10)

In [211]:
from keras.utils import to_categorical
y_train1= to_categorical(y_train1)
y_test1= to_categorical(y_test1)

In [207]:
print(X_train1.shape, y_train1.shape)

(912, 10, 36) (912,)


In [208]:
print(X_test1.shape, y_test1.shape)

(305, 10, 36) (305,)


In [222]:
%%time
from keras.layers.convolutional import Conv1D 
#from keras.layers import Conv1D, MaxPoling1D
def_model = Sequential()
def_model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(10,36)))
#model.add(MaxPooling1D(pool_size=1))
def_model.add(LSTM(100))
def_model.add(Dense(4, activation='softmax'))
def_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(def_model.summary())
def_model.fit(X_train1, y_train1, epochs=10, batch_size=64)
# Final evaluation of the model
scores = def_model.evaluate(X_test1, y_test1, verbose=0)
print(def_model.summary())
# Final evaluation of the model
print("accuracy: %.2f%%" % (scores[1]*100))

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_20 (Conv1D)           (None, 8, 32)             3488      
_________________________________________________________________
lstm_19 (LSTM)               (None, 100)               53200     
_________________________________________________________________
dense_19 (Dense)             (None, 4)                 404       
Total params: 57,092
Trainable params: 57,092
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
15/15 [==============================] - 1s 5ms/step - loss: 1.3482 - accuracy: 0.5801
Epoch 2/10
15/15 [==============================] - 0s 4ms/step - loss: 0.8903 - accuracy: 0.7193
Epoch 3/10
15/15 [==============================] - 0s 4ms/step - loss: 0.7317 - accuracy: 0.7350
Epoch 4/10
15/15 [==============================] - 0s 4ms/step - loss: 0.6798 -

In [223]:
!mkdir -p saved_model
def_model.save('saved_model/def_model')

INFO:tensorflow:Assets written to: saved_model/def_model/assets


INFO:tensorflow:Assets written to: saved_model/def_model/assets


In [227]:
!mkdir -p saved_model
off_model.save('saved_model/off_model')

INFO:tensorflow:Assets written to: saved_model/off_model/assets


INFO:tensorflow:Assets written to: saved_model/off_model/assets


In [228]:
!ls saved_model


def_model  model  off_model


In [229]:
new_model = tf.keras.models.load_model('saved_model/off_model')


In [230]:

# save numpy array as csv file
from numpy import asarray
from numpy import savetxt
# define data
#data = asarray([[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]])
# save to csv file
savetxt('off_X_train.csv', X_train, delimiter=',')
# save numpy array as csv file

ValueError: ignored

In [244]:
# reshaping the array from 3D
# matrice to 2D matrice.
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)
y_train_reshaped = y_train.reshape(y_train.shape[0], -1)
y_test_reshaped = y_test.reshape(y_test.shape[0], -1)
X_train1_reshaped = X_train1.reshape(X_train1.shape[0], -1)
X_test1_reshaped = X_test1.reshape(X_test1.shape[0], -1)
y_train1_reshaped = y_train1.reshape(y_train1.shape[0], -1)
y_test1_reshaped = y_test1.reshape(y_test1.shape[0], -1)

In [245]:
# saving reshaped array to file.
np.savetxt("X_train.csv", X_train_reshaped)
np.savetxt("X_test.csv", X_test_reshaped)
np.savetxt("y_train.csv", y_train_reshaped)
np.savetxt("y_test.csv", y_test_reshaped)
np.savetxt("X_train1.csv", X_train1_reshaped)
np.savetxt("X_test1.csv", X_test1_reshaped)
np.savetxt("y_train1.csv", y_train1_reshaped)
np.savetxt("y_test1.csv", y_test1_reshaped)


In [237]:
# retrieving data from file.
loaded_X_train = np.loadtxt("X_train.csv")

In [238]:
# matrice with original shape.
load_original_arr = loaded_X_train.reshape(
    loaded_X_train.shape[0], loaded_X_train.shape[1] // X_train.shape[2], X_train.shape[2])
  

In [239]:
# check the shapes:
print("shape of arr: ", X_train.shape)
print("shape of load_original_arr: ", load_original_arr.shape)

shape of arr:  (910, 10, 36)
shape of load_original_arr:  (910, 10, 36)
